In [1]:
from data_sources.stock_fetcher import StockDataFetcher

In [2]:
# Enhanced stock fetcher for experimentation
import logging
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ExperimentalStockFetcher:
    """Enhanced fetcher for testing different periods."""
    
    def __init__(self, period: str = "3mo"):
        self.period = period
        
    def fetch_single_with_period(self, symbol: str, period: str = None) -> pd.DataFrame:
        """Fetch data with custom period."""
        period = period or self.period
        ticker = yf.Ticker(symbol)
        
        print(f"Fetching {symbol} for period: {period}")
        hist = ticker.history(period=period)
        
        if hist.empty:
            raise ValueError(f"No data returned for {symbol}")
            
        hist['symbol'] = symbol
        hist = hist.reset_index()  # Make date a column
        
        print(f"Got {len(hist)} rows of data for {symbol}")
        return hist
    
    def test_periods(self, symbol: str = "AAPL"):
        """Test different time periods."""
        periods = ["1mo", "3mo", "6mo", "1y"]
        results = {}
        
        for period in periods:
            try:
                data = self.fetch_single_with_period(symbol, period)
                results[period] = {
                    'rows': len(data),
                    'date_range': f"{data['Date'].min().date()} to {data['Date'].max().date()}",
                    'latest_price': f"${data['Close'].iloc[-1]:.2f}"
                }
            except Exception as e:
                results[period] = {'error': str(e)}
                
        return results
    
    def check_indicator_requirements(self, data: pd.DataFrame):
        """Check if we have enough data for technical indicators."""
        rows = len(data)
        
        return {
            "total_rows": rows,
            "rsi_ready": rows >= 15,  # Need 14+ for RSI
            "macd_ready": rows >= 35,  # Need 26 + 12 for MACD  
            "sma_50_ready": rows >= 50,  # Need 50 for SMA50
            "recommendation": "Need 3mo+" if rows < 50 else "✅ Sufficient data"
        }

# Create the experimental fetcher
exp_fetcher = ExperimentalStockFetcher()

In [3]:
# Test different periods for AAPL
print("=== Testing Different Periods ===")
results = exp_fetcher.test_periods("AAPL")

for period, data in results.items():
    print(f"\n📊 {period.upper()}:")
    if 'error' in data:
        print(f"   ❌ Error: {data['error']}")
    else:
        print(f"   📈 Rows: {data['rows']}")
        print(f"   📅 Range: {data['date_range']}")
        print(f"   💰 Latest: {data['latest_price']}")

=== Testing Different Periods ===
Fetching AAPL for period: 1mo
Got 24 rows of data for AAPL
Fetching AAPL for period: 3mo
Got 64 rows of data for AAPL
Fetching AAPL for period: 6mo
Got 125 rows of data for AAPL
Fetching AAPL for period: 1y
Got 251 rows of data for AAPL

📊 1MO:
   📈 Rows: 24
   📅 Range: 2025-07-15 to 2025-08-15
   💰 Latest: $231.41

📊 3MO:
   📈 Rows: 64
   📅 Range: 2025-05-15 to 2025-08-15
   💰 Latest: $231.41

📊 6MO:
   📈 Rows: 125
   📅 Range: 2025-02-18 to 2025-08-15
   💰 Latest: $231.41

📊 1Y:
   📈 Rows: 251
   📅 Range: 2024-08-15 to 2025-08-15
   💰 Latest: $231.41


In [4]:
# Get 3mo data and check indicator requirements
print("=== Checking Data Requirements ===")
data_3mo = exp_fetcher.fetch_single_with_period("AAPL", "3mo")
requirements = exp_fetcher.check_indicator_requirements(data_3mo)

print(f"\n📋 Data Analysis:")
for key, value in requirements.items():
    print(f"   {key}: {value}")

print(f"\n📊 Sample Data:")
print(data_3mo[['Date', 'Close', 'Volume']].head(3))
print("...")
print(data_3mo[['Date', 'Close', 'Volume']].tail(3))

=== Checking Data Requirements ===
Fetching AAPL for period: 3mo
Got 64 rows of data for AAPL

📋 Data Analysis:
   total_rows: 64
   rsi_ready: True
   macd_ready: True
   sma_50_ready: True
   recommendation: ✅ Sufficient data

📊 Sample Data:
                       Date       Close    Volume
0 2025-05-15 00:00:00-04:00  211.210297  45029500
1 2025-05-16 00:00:00-04:00  211.020508  54737900
2 2025-05-19 00:00:00-04:00  208.543320  46140500
...
                        Date       Close    Volume
61 2025-08-13 00:00:00-04:00  233.330002  69878500
62 2025-08-14 00:00:00-04:00  232.779999  51857600
63 2025-08-15 00:00:00-04:00  231.360001  37724379


In [5]:
# Test incremental fetching logic
print("=== Testing Incremental Fetching ===")

# Simulate current engine behavior
print("🔄 Current engine.py behavior:")
print("   - Always fetches period='1mo' (last 30 days)")
print("   - Upserts to database (overwrites same dates)")
print("   - Result: No historical accumulation beyond 1 month")

# Show better approach
print("\n✅ Better incremental approach:")
print("   1. Check last date in database for each symbol")
print("   2. Fetch only NEW data since that date") 
print("   3. Append new rows (no overwrites)")
print("   4. Accumulate historical data over time")

print(f"\n📈 For RSI/MACD, you need:")
print(f"   - RSI: 14+ days (current 1mo = ~20-22 trading days ✅)")
print(f"   - MACD: 26+ days (need 3mo = ~65 trading days ✅)")
print(f"   - SMA50: 50+ days (need 3mo+ ✅)")

print(f"\n🎯 Recommendation:")
print(f"   - Change engine.py to use period='3mo' initially")
print(f"   - Then implement incremental daily fetching")
print(f"   - This gives you enough data for all indicators")

=== Testing Incremental Fetching ===
🔄 Current engine.py behavior:
   - Always fetches period='1mo' (last 30 days)
   - Upserts to database (overwrites same dates)
   - Result: No historical accumulation beyond 1 month

✅ Better incremental approach:
   1. Check last date in database for each symbol
   2. Fetch only NEW data since that date
   3. Append new rows (no overwrites)
   4. Accumulate historical data over time

📈 For RSI/MACD, you need:
   - RSI: 14+ days (current 1mo = ~20-22 trading days ✅)
   - MACD: 26+ days (need 3mo = ~65 trading days ✅)
   - SMA50: 50+ days (need 3mo+ ✅)

🎯 Recommendation:
   - Change engine.py to use period='3mo' initially
   - Then implement incremental daily fetching
   - This gives you enough data for all indicators


In [6]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# Historical daily data
data = msft.history(period="6mo", interval="1d")
print(data.head())

# Company info
print(msft.info['sector'], msft.info['marketCap'])

# Dividends
print(msft.dividends)

# Intraday (last 5 days, 5-minute candles)
intraday = msft.history(period="5d", interval="5m")

                                 Open        High         Low       Close  \
Date                                                                        
2025-02-18 00:00:00-05:00  406.437403  409.027451  404.943148  408.071136   
2025-02-19 00:00:00-05:00  406.317861  413.898701  406.088731  413.181458   
2025-02-20 00:00:00-05:00  414.529005  418.541628  411.784044  415.367462   
2025-02-21 00:00:00-05:00  416.575219  417.283909  407.142554  407.461945   
2025-02-24 00:00:00-05:00  407.761419  408.619829  398.588257  403.259674   

                             Volume  Dividends  Stock Splits  
Date                                                          
2025-02-18 00:00:00-05:00  21423100       0.00           0.0  
2025-02-19 00:00:00-05:00  24114200       0.00           0.0  
2025-02-20 00:00:00-05:00  23508700       0.83           0.0  
2025-02-21 00:00:00-05:00  27524800       0.00           0.0  
2025-02-24 00:00:00-05:00  26443700       0.00           0.0  
Technology 38871763

In [8]:
import yfinance as yf
import talib

# Download daily data for last 6 months
data = yf.download("MSFT", period="6mo", interval="1d")



/var/folders/_f/cmvp2vmn60j686_xhfbjc4hm0000gn/T/ipykernel_56807/2862285279.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("MSFT", period="6mo", interval="1d")
[*********************100%***********************]  1 of 1 completed


In [9]:
data

Price,Close,High,Low,Open,Volume
Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
Date,,,,,
2025-02-18,408.071106,409.027421,404.943117,406.437373,21423100
2025-02-19,413.181427,413.898671,406.088701,406.317831,24114200
2025-02-20,415.367462,418.541628,411.784044,414.529005,23508700
2025-02-21,407.461945,417.283909,407.142554,416.575219,27524800
2025-02-24,403.259674,408.619829,398.588257,407.761419,26443700
...,...,...,...,...,...
2025-08-11,521.770020,527.590027,519.719971,522.299988,20194400
2025-08-12,529.239990,530.979980,522.700012,523.750000,18667000


In [ ]:
# Example for MSFT
close_msft = data['Close']['MSFT']

# RSI (14-day)
data['RSI'] = talib.RSI(close_msft.values, timeperiod=14)

# MACD (12,26,9)
data['MACD'], data['MACD_signal'], data['MACD_hist'] = talib.MACD(close_msft.values, fastperiod=12, slowperiod=26, signalperiod=9)



Price            Close        High         Low        Open    Volume  \
Ticker            MSFT        MSFT        MSFT        MSFT      MSFT   
Date                                                                   
2025-08-11  521.770020  527.590027  519.719971  522.299988  20194400   
2025-08-12  529.239990  530.979980  522.700012  523.750000  18667000   
2025-08-13  520.580017  532.700012  519.369995  532.109985  19619200   
2025-08-14  522.479980  525.950012  520.140015  522.559998  20269100   
2025-08-15  523.309998  526.099976  520.859985  522.700012  13178149   

Price             RSI      MACD MACD_signal MACD_hist  
Ticker                                                 
Date                                                   
2025-08-11  59.414272  8.500387    9.473698 -0.973312  
2025-08-12  64.338141  8.547594    9.288477 -0.740883  
2025-08-13  55.874976  7.796348    8.990051 -1.193704  
2025-08-14  57.205033  7.270481    8.646137 -1.375656  
2025-08-15  57.803430  6.841835

In [4]:
import yfinance as yf
from backtesting import Strategy, Backtest

# Download last 10 years of daily MSFT data
msft = yf.Ticker("MSFT")
data = msft.history(period="10y", interval="1d")
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]


In [6]:
from backtesting import Backtest, Strategy
import talib
import numpy as np

class TrendPullbackRSI2(Strategy):
    # ---- Tunable params (safe, low-variance ranges) ----
    sma_len = 200          # long-term trend filter
    rsi_len = 2            # short RSI for mean-reversion entry
    rsi_buy = 10           # buy when RSI2 <= this (deep dip)
    atr_len = 14           # ATR for risk sizing
    atr_mult = 3.0         # stop/trailing distance in ATRs
    ce_lookback = 22       # chandelier lookback (highest high window)
    risk_per_trade = 0.01  # risk 1% of equity per position

    def init(self):
        c = self.data.Close
        h = self.data.High
        l = self.data.Low

        self.sma200 = self.I(talib.SMA, c, self.sma_len)
        self.rsi2   = self.I(talib.RSI, c, self.rsi_len)
        self.atr    = self.I(talib.ATR, h, l, c, self.atr_len)

    def next(self):
        price = self.data.Close[-1]
        sma200_now = self.sma200[-1]
        sma200_prev = self.sma200[-2] if len(self.sma200) > 1 else sma200_now
        atr = self.atr[-1]

        # Uptrend filter: price above rising SMA200
        in_uptrend = (price > sma200_now) and (sma200_now > sma200_prev)

        if not self.position:
            # Buy a sharp dip within the uptrend
            if in_uptrend and self.rsi2[-1] <= self.rsi_buy and atr > 0:
                # Volatility-based position sizing
                risk_per_share = self.atr_mult * atr
                size = max(1, int((self.equity * self.risk_per_trade) / max(risk_per_share, 1e-9)))

                # Initial protective stop at entry - k*ATR
                sl = price - risk_per_share
                self.buy(size=size, sl=sl)
        else:
            # Chandelier trailing stop: highest high over N bars minus k*ATR
            lookback = int(self.ce_lookback)
            hh = float(np.max(self.data.High[-lookback:])) if len(self.data.High) >= lookback else self.data.High.max()
            trail = hh - self.atr_mult * atr

            # Exit if trend breaks or trail is hit
            if price < trail or price < sma200_now:
                self.position.close()


In [7]:
from backtesting import Backtest, Strategy
import numpy as np
import talib

class AdaptiveRSI_MACD_Z(Strategy):
    # --- knobs you can tune coarsely ---
    sma_len = 200             # trend filter
    rsi_len = 14
    macd_fast, macd_slow, macd_sig = 12, 26, 9
    stat_win = 63             # rolling window (≈ 3 months) for mean/std
    z_buy_rsi  = -1.5         # buy if RSI z <= this
    z_buy_hist = -0.5         # and MACD-hist z <= this (oversold momentum)
    z_exit_rsi =  0.5         # exit if RSI z >= this (mean reversion complete)
    z_exit_hist=  0.0         # or MACD-hist z >= this (momentum back to neutral+)
    atr_len = 14
    atr_mult_stop = 3.0       # fail-safe stop distance in ATR
    risk_per_trade = 0.01     # risk 1% of equity per entry
    min_hold = 3              # bars
    max_hold = 20             # bars
    cooldown_bars = 5         # wait after full exit

    def init(self):
        c, h, l = self.data.Close, self.data.High, self.data.Low
        # Trend
        self.sma = self.I(talib.SMA, c, self.sma_len)

        # RSI and its rolling z-score
        self.rsi = self.I(talib.RSI, c, self.rsi_len)
        self.rsi_z = self.I(lambda x, w:
                            (x - talib.SMA(x, w)) / (talib.STDDEV(x, w) + 1e-9),
                            self.rsi, self.stat_win)

        # MACD histogram and its rolling z-score
        self.macd, self.signal, self.hist = self.I(talib.MACD, c,
                                                   self.macd_fast, self.macd_slow, self.macd_sig)
        self.hist_z = self.I(lambda x, w:
                             (x - talib.SMA(x, w)) / (talib.STDDEV(x, w) + 1e-9),
                             self.hist, self.stat_win)

        # ATR for stops/sizing
        self.atr = self.I(talib.ATR, h, l, c, self.atr_len)

        # bookkeeping
        self.entry_i = None
        self.last_full_exit_i = -10**9  # far past

    def _in_uptrend(self):
        sma_now  = float(self.sma[-1])
        sma_prev = float(self.sma[-2]) if len(self.sma) > 1 else sma_now
        return float(self.data.Close[-1]) > sma_now and sma_now > sma_prev

    def _enough_history(self):
        # ensure rolling stats are valid
        vals = [self.rsi_z[-1], self.hist_z[-1], self.atr[-1], self.sma[-1]]
        return all(np.isfinite(v) for v in vals)

    def next(self):
        if not self._enough_history():
            return

        price = float(self.data.Close[-1])
        atr   = max(float(self.atr[-1]), 1e-9)
        i     = self.i

        # --------- EXIT rules (evaluate first if in a trade) ---------
        if self.position:
            # time stop (but allow min_hold first)
            held = i - (self.entry_i if self.entry_i is not None else i)
            can_exit = held >= self.min_hold

            # trend break → exit immediately
            if not self._in_uptrend():
                self.position.close()
                self.entry_i = None
                self.last_full_exit_i = i
                return

            # ATR fail-safe trail (discrete check)
            lookback = 22
            hh = float(max(self.data.High[-lookback:]))
            trail = hh - self.atr_mult_stop * atr
            if can_exit and price < trail:
                self.position.close()
                self.entry_i = None
                self.last_full_exit_i = i
                return

            # z-score exits (mean reversion achieved)
            if can_exit and (float(self.rsi_z[-1]) >= self.z_exit_rsi or
                             float(self.hist_z[-1]) >= self.z_exit_hist):
                self.position.close()
                self.entry_i = None
                self.last_full_exit_i = i
                return

            # hard time stop
            if held >= self.max_hold:
                self.position.close()
                self.entry_i = None
                self.last_full_exit_i = i
                return

        # --------- ENTRY rules (only if flat) ---------
        if not self.position:
            # cooldown after full exit
            if i - self.last_full_exit_i < self.cooldown_bars:
                return

            # Uptrend + deep relative oversold on both RSI and MACD-hist
            if self._in_uptrend() and \
               float(self.rsi_z[-1]) <= self.z_buy_rsi and \
               float(self.hist_z[-1]) <= self.z_buy_hist and \
               float(self.hist[-1]) > float(self.hist[-2]):  # momentum turning up
                # ATR risk-based size
                risk_per_share = self.atr_mult_stop * atr
                if risk_per_share > 0:
                    risk_capital = self.equity * self.risk_per_trade
                    size = int(max(0, risk_capital / risk_per_share))
                    if size > 0:
                        sl = price - self.atr_mult_stop * atr
                        self.buy(size=size, sl=sl)
                        self.entry_i = i

# ---- Run example (fold any slippage into commission) ----
# bt = Backtest(data, AdaptiveRSI_MACD_Z, cash=100_000, commission=0.001, exclusive_orders=True)
# stats = bt.run(); print(stats); bt.plot()


In [8]:
bt = Backtest(data, AdaptiveRSI_MACD_Z, cash=100_000, commission=0.001, exclusive_orders=True)
stats = bt.run(); print(stats); bt.plot()


AttributeError: 'AdaptiveRSI_MACD_Z' object has no attribute 'i'

In [32]:
bt = Backtest(
    data,
    CorePlusDips,
    cash=100_000,
    commission=0.0005,
    slippage=0.0005,
    exclusive_orders=True
)
stats = bt.run()
print(stats)
bt.plot()


TypeError: Backtest.__init__() got an unexpected keyword argument 'slippage'

In [ ]:
from bokeh.io import output_notebook

output_notebook()   # This enables inline plotting in Jupyter


bt.plot()

Loading BokehJS ...

/Users/youngmincho/Project/araverus/quant/quant-ve/lib/python3.12/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p4124', ...)